In [ ]:
! pip install datasets
! pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9784a141b1dea9f531312f0ee76ac7aa16023ece1de365e0704bf9cf639d8ade
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from evaluation_suite import EvaluationSuite

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import pprint

from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

import torch

from peft import LoraConfig, get_peft_model, TaskType

import json
import pprint
import pprint
# Import weights and biases
import wandb
# Import kaggle secrets
from google.colab import userdata

# Load data

In [ ]:
with open('train_dataset.json', 'r') as f:
    train_dataset = json.load(f)

# Load test_dataset
with open('test_dataset.json', 'r') as f:
    test_dataset = json.load(f)

multiple_choice_testset = Dataset.from_list(test_dataset['multiple_choice'])
multiple_choice_trainset = Dataset.from_list(train_dataset['multiple_choice'])

In [ ]:
# Define a system prompt under prompt_style
prompt_style_multiple_choice = """
"{_question_var_}"

### What is the correct answer? Please state only the letter:
"""

def build_prompt(prompt_style: str, question: str):
  return prompt_style.replace("{_question_var_}", question)

In [ ]:
def generate_model_input(example):
    question = example['question']
    options = example['options']

    model_input = question + '\nOptions:\n'
    for key, val in options.items():
        model_input += f"{key}. {val}\n"

    example['model_input'] = build_prompt(prompt_style_multiple_choice, model_input)
    return example

In [ ]:
multiple_choice_testset = multiple_choice_testset.map(generate_model_input)
multiple_choice_trainset = multiple_choice_trainset.map(generate_model_input)

Map:   0%|          | 0/43466 [00:00<?, ? examples/s]

Map:   0%|          | 0/173861 [00:00<?, ? examples/s]

In [ ]:
def format(example):
    question = example["model_input"]
    output = example["correct_answer"]

    full_text = question + output

    return {"prompt": question, "output": output, "text": full_text}


In [ ]:
trainset = multiple_choice_trainset.map(format)
testset = multiple_choice_testset.map(format)

Map:   0%|          | 0/173861 [00:00<?, ? examples/s]

Map:   0%|          | 0/43466 [00:00<?, ? examples/s]

In [ ]:
evalset = trainset.train_test_split(test_size=0.0005)
trainset = evalset["train"]
evalset = evalset["test"]

# Fine-tuning

In [ ]:
model_id = "deepseek-ai/deepseek-coder-7b-instruct-v1.5"
project_name = "CinIQ_fine-tuning"
run_name = "ft_multiple_choice_v1_early_stopping"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/621 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 3,932,160 || all params: 6,914,297,856 || trainable%: 0.0569


In [ ]:
wandb.init(project=project_name, name=run_name)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: konstantinwehmeyer (konstantinwehmeyer-university-of-st-gallen-student-union) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
def tokenize(example):
    prompt_ids = tokenizer(
        example["prompt"],
        truncation=True,
        max_length=1024, # reduced to avoid RAM overflow
        padding="max_length"
    )

    full_ids = tokenizer(
        example["text"],
        truncation=True,
        max_length=1024,
        padding="max_length"
    )

    labels = full_ids["input_ids"].copy()

    # Mask the prompt tokens — we only want to train on the output
    prompt_len = len(tokenizer(example["prompt"])["input_ids"])
    labels[:prompt_len] = [-100] * prompt_len  # -100 = ignore index for loss

    full_ids["labels"] = labels
    return full_ids


In [ ]:
tokenized = trainset.map(tokenize, batched=True)
# tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"]) # what is the right one??
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/154645 [00:00<?, ? examples/s]

In [ ]:
tokenized_evalset = evalset.map(tokenize, batched=True)
tokenized_evalset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./deepseek-coder-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_dir="./logs",
    logging_steps=5,
    save_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # or "accuracy" if you log it
    greater_is_better=False,
    report_to="wandb",  # W&B tracking
    run_name=run_name
)


In [ ]:
model.gradient_checkpointing_disable()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    eval_dataset=tokenized_evalset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

<ipython-input-36-5e27b97758c0>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
50,1.347700,1.319907
100,1.344100,1.306544
150,1.376300,1.301262
200,1.438200,1.298016
250,1.415400,1.288813
300,1.332400,1.288799
350,1.330500,1.282267
400,1.340200,1.276784
450,1.368600,1.272275
500,1.339300,1.267896


Step,Training Loss,Validation Loss
50,1.347700,1.319907
100,1.344100,1.306544
150,1.376300,1.301262
200,1.438200,1.298016
250,1.415400,1.288813
300,1.332400,1.288799
350,1.330500,1.282267
400,1.340200,1.276784
450,1.368600,1.272275
500,1.339300,1.267896


In [ ]:
model.save_pretrained(f"./{run_name}")
tokenizer.save_pretrained(f"./{run_name}")

# Load fine-tuned model

In [ ]:
model_path = f"./{run_name}"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)

# Evaluate fine-tuned model

In [ ]:
multiple_choice_testset = test_dataset['multiple_choice']

for sample in multiple_choice_testset:
  question = sample.get('question')
  options = sample.get('options')
  model_input = question + '\n Options: '
  for option in options:
    model_input += option + '. ' +options[option] + '\n'

  multiple_choice_testset[multiple_choice_testset.index(sample)]['model_input'] = model_input

In [ ]:
# Define a system prompt under prompt_style
prompt_style_multiple_choice = """
"{_question_var_}"

### What is the correct answer? Please state only the letter:
"""

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
response = pipe(build_prompt(prompt_style_multiple_choice, multiple_choice_testset[0]['model_input']))

In [ ]:
pprint.pp(response)
print(multiple_choice_testset[0]['correct_answer'])

In [ ]:
predictions_multiple_choice = []

for i in range(50):

  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  response = pipe(build_prompt(prompt_style_multiple_choice, multiple_choice_testset[i]['model_input']))

  predictions_multiple_choice.append(response)

In [ ]:
import re
predicted_answer_multiple_choice = []
for sample in predictions_multiple_choice:
  match = re.search(r'Please state only the letter:\s*\n*([A-E])\.*', sample[0].get('generated_text'))
  if match:
      answer = match.group(1)
      predicted_answer_multiple_choice.append(answer)
  else:
      predicted_answer_multiple_choice.append("na")

ground_truth = [entry['correct_answer'] for entry in multiple_choice_testset[:50]]

In [ ]:
ground_truth = [entry['correct_answer'] for entry in multiple_choice_testset[:50]]

evaluation_suite = EvaluationSuite()

evaluation_suite.evaluate_MC(predicted_answer_multiple_choice, ground_truth)